# Test onnx/pmml model usage

In [ ]:
SPORT = "nfl"
SERVICE = "draftkings"
TARGET = "top-score"
MODEL_TYPE = 'tpot'
CONTEST_TYPE = "FIFTY_FIFTY"
STYLE = "CLASSIC"
FORMAT = "pmml"

MODEL_DIR = "eval_results"
DATA_FOLDER = "data"
RANDOM_STATE = 0


In [ ]:

import os

import onnx
from pypmml import Model


def load_model(sport, service, style, contest_type, target, model_type, model_path, format):
    """ load an onnx model for use """
    print(f"loading model for {sport=} {service=} {contest_type=} {style=} {target=} {model_type=} {format=}")
    filepath = os.path.join(model_path, f"{sport}-{service}-{style}-{contest_type}-{target}-{model_type}.{format}")
    if format == 'onnx':
        model = onnx.load(filepath)
        onnx.checker.check_model(model)
        # print("inputs: ", [input.name for input in model.graph.input])
        # print("outputs: ", [output.name for output in model.graph.output])    
    else:
        model = Model.load(filepath)
    return model

In [ ]:
from generate_train_test import generate_train_test, load_csv
from automl import error_report
import pandas as pd

df = load_csv(SPORT, SERVICE, STYLE, CONTEST_TYPE, data_folder=DATA_FOLDER)
display(df)
model_data = generate_train_test(
    df,
    random_state=RANDOM_STATE,
)
X_test: pd.DataFrame
(X_train, X_test, y_top_train, y_top_test,
    y_last_win_train, y_last_win_test) = model_data
y_test = y_top_test if TARGET == "top-score" else y_last_win_test

In [ ]:
import numpy as np

model = load_model(SPORT, SERVICE, STYLE, CONTEST_TYPE, TARGET, MODEL_TYPE, MODEL_DIR, FORMAT)
print(f"{model=}")
# error_report(
#     model, X_test, y_test, 
#     desc=f"{SPORT=} {SERVICE=} {STYLE=} {CONTEST_TYPE=} {TARGET=} {MODEL_TYPE=} {FORMAT=}"
# )
predictions = model.predict(X_test)
display("model predictions:", predictions)
# display("x:", X_test)
# display("y:", y_test)
nans = pd.isna(predictions[predictions.columns[0]])
np.count_nonzero(nans)